In [13]:
import json
import sys
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import wandb
from ignite.contrib.handlers import wandb_logger
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.handlers.param_scheduler import LRScheduler
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from src.datasets import FordDataset
from src.models import TransformerClassification
from src.utils import build_optimizer, str2torch

In [14]:
with open('../configs/transformer_87.json') as f:
    config =  json.load(f)

In [15]:
train_path = "../data/FordA/FordA_TRAIN.arff"
test_path = "../data/FordA/FordA_TEST.arff"

train_dataset = FordDataset(train_path, config['data'])
test_dataset = FordDataset(test_path, config['data'])

idx = np.arange(len(train_dataset))
idx_train, idx_val = train_test_split(idx, train_size=0.8, stratify=train_dataset.labels, random_state=config['random_state'])

train_sampler = SubsetRandomSampler(idx)
val_sampler = SubsetRandomSampler(idx_val)

train_dataloader = DataLoader(train_dataset, batch_size=config['data']['batch_size'], sampler=train_sampler)
val_dataloader = DataLoader(train_dataset, batch_size=config['data']['batch_size'], sampler=val_sampler)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [23]:
# Initialize your model
wandb.init(entity='ts-robustness', project='ml-course', config=config, tags=['hypersearch'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
config['train']['optimizer'] = str2torch(config['train']['optimizer'])

model = TransformerClassification(config).to(device)

# Initialize your optimizer and criterion
optimizer = build_optimizer(config, model)
criterion = nn.CrossEntropyLoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.long())
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

train_evaluator = Engine(validation_step)
val_evaluator = Engine(validation_step)
test_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (torch.argmax(x[0], dim=1), x[1])),
    'loss': Loss(criterion, output_transform=lambda x: (x[0], x[1].long()))
}

for name, metric in metrics.items():
    metric.attach(train_evaluator, name)
    metric.attach(val_evaluator, name)
    metric.attach(test_evaluator, name)


# checkpoint_handler = ModelCheckpoint(dirname='saved_models', filename_prefix='best',
#                                      n_saved=1, require_empty=False,
#                                      score_function=lambda engine: engine.state.metrics['accuracy'],
#                                      score_name="accuracy", global_step_transform=lambda *_: trainer.state.epoch)
# val_evaluator.add_event_handler(Events.EPOCH_COMPLETED, checkpoint_handler, {"model": model})


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"train_accuracy": metrics['accuracy'],
               "train_loss": metrics['loss']})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_dataloader)
    metrics = val_evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"val_accuracy": metrics['accuracy'],
               "val_loss": metrics['loss']})
    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_dataloader)
    metrics = test_evaluator.state.metrics
    print("Test Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['loss']))
    wandb.log({"test_accuracy": metrics['accuracy'],
               "test_loss": metrics['loss']})


# Run the training loop
trainer.run(train_dataloader, max_epochs=config['train']['n_epoch'])
wandb.finish()

Training Results - Epoch: 1  Avg accuracy: 0.69 Avg loss: 0.6025
Validation Results - Epoch: 1  Avg accuracy: 0.69 Avg loss: 0.6019
Training Results - Epoch: 2  Avg accuracy: 0.81 Avg loss: 0.4264
Validation Results - Epoch: 2  Avg accuracy: 0.81 Avg loss: 0.4233
Training Results - Epoch: 3  Avg accuracy: 0.84 Avg loss: 0.3639
Validation Results - Epoch: 3  Avg accuracy: 0.84 Avg loss: 0.3572
Training Results - Epoch: 4  Avg accuracy: 0.85 Avg loss: 0.3424
Validation Results - Epoch: 4  Avg accuracy: 0.86 Avg loss: 0.3341
Training Results - Epoch: 5  Avg accuracy: 0.86 Avg loss: 0.3168
Validation Results - Epoch: 5  Avg accuracy: 0.86 Avg loss: 0.3065
Training Results - Epoch: 6  Avg accuracy: 0.87 Avg loss: 0.3005
Validation Results - Epoch: 6  Avg accuracy: 0.88 Avg loss: 0.2894
Training Results - Epoch: 7  Avg accuracy: 0.88 Avg loss: 0.2959
Validation Results - Epoch: 7  Avg accuracy: 0.89 Avg loss: 0.2854
Training Results - Epoch: 8  Avg accuracy: 0.87 Avg loss: 0.2964
Validation 

test_accuracy,▁
test_loss,▁
train_accuracy,▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███▇█▇███████
train_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▂▁▁▁▁▁▁
val_accuracy,▁▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇████▇█▇▇████▇█
val_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▂▂▁▁▁▁▁▁
test_accuracy,0.8697
test_loss,0.31976
train_accuracy,0.90107
train_loss,0.23561
val_accuracy,0.90715


In [26]:
torch.save(model.to('cpu').state_dict(), "../models/trans_2outp_87.pth")

In [25]:
# Count the number of parameters in the transformer_encoder layer
transformer_encoder_params = sum(p.numel() for p in model.transformer_encoder.parameters())

# Count the number of parameters in the fc layer
fc_params = sum(p.numel() for p in model.fc.parameters())

print("Number of parameters in transformer_encoder:", transformer_encoder_params)
print("Number of parameters in fc layer:", fc_params)


Number of parameters in transformer_encoder: 214346
Number of parameters in fc layer: 187652
